In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.spatial.distance import cosine, cdist
import numpy as np

In [ ]:
import os

vv = TfidfVectorizer()

docs = [] # a list of the text content of every document
names = [] # maintains a list of file names
for root, dirs, files in os.walk('cran/docs'):
    for file in files[:1400]:
        with open(os.path.join(root,file), 'r') as f:
            docs.append(f.read())
            names.append(file)
            
vv.fit(docs)

transformed = vv.transform(docs)
print(transformed.shape) 
print(transformed.toarray())

In [ ]:

queries = [] # a list of the text content of every query
qnames = [] # maintains a list of file names
for root, dirs, files in os.walk('cran/queries'):
    for file in files[:225]:
        with open(os.path.join(root,file), 'r') as f:
            queries.append(f.read())
            qnames.append(file)

transformedQueries = vv.transform(queries)
print(transformedQueries.shape) 
print(transformedQueries.toarray()) 

In [ ]:
import numpy as np
sorted = []
sims = 1-cdist(transformedQueries.toarray(), transformed.toarray(), metric='cosine')
for x in sims:
    sorted.append(np.argsort(x))
type (sims)
max_pos = np.argmax(sorted)


In [ ]:
i = 1
for d in sorted:
    print('Query',i,'has\n')
    print (d,'as relevant documents indices')
    i = i + 1
#     for i in d:
#         print('\nFile:\n',i+1)
#         print(open('cran/docs/'+names[i], 'r').read())

In [ ]:
for d in sorted:
    for s in d:
        print('\nFile:',names[s],'\n') #printing docs sorted by the relevance
        print(open('cran/docs/'+names[s], 'r').read())

In [ ]:
actualResults = {}
with open('cran/cranqrel') as f:
    for l in f:
        words = l.split()
        query = int(words[0])
        if query not in actualResults:
            actualResults[query] = []
        
        actualResults[query].append(int(words[1]))

In [ ]:
def manual_pr(ref, res):
    
    intersect = len(set(ref).intersection(set(res)))
    precision = intersect/len(res)

    return precision


In [ ]:
i = 1
totalPrec = 0
while i < 225:
    r = sorted[i]
    ar = actualResults[i]
    prec = manual_pr(ar,r)
    totalPrec =  totalPrec + prec
    i=i+1
    print(totalPrec)
    
print('MAP=', totalPrec/225)